In [2]:
import os
import sys

import adanet
import tensorflow as tf
from tensorflow import keras

In [3]:
data = tf.keras.datasets.cifar100.load_data()

train = data[0]
test = data[1]

train_image, train_label = train[0].astype(np.float32), train[1]
test_image, test_label= test[0].astype(np.float32), test[1]

train_label = np.reshape(train_label, newshape=[-1])
test_label = np.reshape(test_label, newshape=[-1])

# Generate Dataset obj   
dataset_obj = tf.data.Dataset.from_tensors(
    {'image': train_image, 'label': train_label})
dataset_obj = dataset_obj.shuffle(50000)
dataset_obj = dataset_obj.unbatch()

# split train-validation dataset
train_dataset = dataset_obj.take(40000)
val_dataset = dataset_obj.skip(40000).take(10000)

test_dataset = tf.data.Dataset.from_tensors(
  {'image': test_image, 'label': test_label})
test_dataset = test_dataset.shuffle(10000).unbatch()

def _preprocessing(dataset, train_mode):
  """
  While train steps, image will be padded random crop and filped(horizontaly)
  And entire steps, per-pixel mean subtracted will be required.
  Args:
    dataset: 'tf.data.Dataset'
    train_mode: 'bool'
  Returns:
    'tf.data.Dataset'
  """
  if train_mode:
    image = dataset['image']
    pad = tf.constant([[2, 2], [2, 2], [0, 0]])
    image = tf.pad(
      tensor=image, paddings=pad)
    image = tf.image.random_crop(
      value=image, size=[32, 32, 3])
    image = tf.image.random_flip_left_right(image=image)
  else:
    image = dataset['image']
    
  label = dataset['label']
  return (image, label)

train_dataset = train_dataset.map(
  lambda x: _preprocessing(x, train_mode=True))
val_dataset = val_dataset.map(
  lambda x: _preprocessing(x, train_mode=False))
test_dataset = test_dataset.map(
  lambda x: _preprocessing(x, train_mode=False))


train_dataset = train_dataset.repeat()
val_dataset = val_dataset.repeat()

In [4]:
base_path = os.environ['HOME'] + '/Estimator'
if os.path.isdir(base_path):
  pass
else:
  os.mkdir(base_path)
MODEL_DIR = base_path+'/simple_nn_estimator'

In [5]:
def gen_input_fn(ds_obj, batch_size):
  def _make():
    return ds_obj.shuffle(1000).batch(batch_size)
  return _make

train_input = gen_input_fn(ds_obj=train_dataset, batch_size=100)
val_input = gen_input_fn(ds_obj=val_dataset, batch_size=100)

In [6]:
head = tf.estimator.MultiClassHead(
  n_classes=100, loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE)

In [7]:
run_config = tf.estimator.RunConfig(
  model_dir=MODEL_DIR,
  save_summary_steps=100,
  save_checkpoints_steps=100,
  keep_checkpoint_max=5,
  session_creation_timeout_secs=600)

In [8]:
# Training step에서 필요한 model임
@tf.function
def _image_normalizer(image):
  image = tf.reshape(image, shape=[-1, 32, 32, 3])
  image = tf.math.subtract(
    x=image,
    y=tf.reshape(
        tf.math.reduce_mean(image, axis=3),
        shape=[-1, 32, 32, 1]))
  return image

In [9]:
# test model
inputs = keras.Input(shape=[32, 32, 3], name='input')

identity_x = keras.layers.Conv2D(
  filters=64, kernel_size=[1, 1], padding='same')(inputs)

x = keras.layers.Conv2D(
  filters=64, kernel_size=[1, 1], padding='same')(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Conv2D(
  filters=64, kernel_size=[1, 1], padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Conv2D(
  filters=64, kernel_size=[1, 1], padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Add()([x, identity_x])
logit = keras.layers.GlobalAveragePooling2D()(x)
logit = keras.layers.Dense(100)(logit)

In [10]:
model = keras.Model(inputs, logit)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   256         input[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 32, 32, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [42]:
[x for x in dir(model) if 'summary' in x]

['summary']

In [ ]:
class SimpleResidualBlockBuilder(adanet.subnetwork.Builder):
  def __init__(self, learning_rate, max_steps, seed):
    self.LEARNING_RATE = learning_rate
    self.MAX_STEPS = max_steps
    self.SEED = seed
    return
  
  def build_subnetwork(self, features, logits_dimension, training,
                      iteration_step, summary, previous_ensemble=None):
    images = list(features.values())[0]
    
    # generate residual block
    x = 
    
    